In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
# install simpleT5
!pip install simplet5

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.8.1+cu101', '4.6.1', '0.10.3')

In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


##MOH-X - Abstract Summarization (Noun+Verb)

In [ ]:
data_file = pd.read_csv("data/moh-x.csv")

In [ ]:
data_file = data_file.rename(columns={'sentence': 'source_text'}) #abstract - sentence
data_file['target_text'] = data_file[['arg1', 'verb']].agg(' '.join, axis=1) #title - metaphorical words
#data_file['target_text'] = data_file['verb'] #title - metaphorical words

In [ ]:
papers = data_file[["source_text", "target_text"]]
papers.head()

,source_text,target_text
0,He absorbed the knowledge or beliefs of his t...,knowledge absorb
1,He absorbed the costs for the accident .,cost absorb
2,The sales tax is absorbed into the state inco...,tax absorb
3,The immigrants were quickly absorbed into soc...,immigrant absorb
4,Her interest in butterflies absorbs her compl...,interest absorb


In [ ]:
# split the data into training and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(papers, test_size=0.1)

In [ ]:
# import
from simplet5 import SimpleT5

# instatntiate
model = SimpleT5()

# load
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, eval_df=test_df, source_max_token_len=512, target_max_token_len=128, max_epochs=5, batch_size=8, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.core.lightning:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!ls outputs/

config.json			    SimpleT5-epoch-2-train-loss-0.2695
eval_results.txt		    SimpleT5-epoch-3-train-loss-0.1619
merges.txt			    SimpleT5-epoch-3-train-loss-0.1764
model_args.json			    SimpleT5-epoch-4-train-loss-0.1171
pytorch_model.bin		    SimpleT5-epoch-4-train-loss-0.1437
SimpleT5-epoch-0-train-loss-1.4472  special_tokens_map.json
SimpleT5-epoch-0-train-loss-1.915   tokenizer_config.json
SimpleT5-epoch-1-train-loss-0.4197  tokenizer.json
SimpleT5-epoch-1-train-loss-0.5195  training_args.bin
SimpleT5-epoch-2-train-loss-0.2453  vocab.json


In [ ]:
# load a trained model
model.load_model("outputs/SimpleT5-epoch-4-train-loss-1.1577", use_gpu=True)

In [ ]:
# let's see how it performerd:
sample_abstracts = test_df.sample(65)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Original Sentence =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Metaphorical Tokens =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Predicted Metaphorical Tokens =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Original Sentence =====
 The invaders spread their language all over the country .

===== Actual Metaphorical Tokens =====
language spread

===== Predicted Metaphorical Tokens =====
language spread

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
 Her feelings condensed .

===== Actual Metaphorical Tokens =====
feeling condense

===== Predicted Metaphorical Tokens =====
feelings condense

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
 He injected new life into the performance .

===== Actual Metaphorical Tokens =====
life inject

===== Predicted Metaphorical Tokens =====
life inject

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
 He heaped him with work .

===== Actual Metaphorical Tokens =====
work heap

===== Predicted Metaphorical Tokens =====
worker heap

 +++++++++++++++++++++